In [1]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent, AgentType, AgentOutputParser, LLMSingleActionAgent
from langchain_experimental.tools import PythonREPLTool
from langchain.chat_models import ChatOllama
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import re
from langchain.schema import AgentAction, AgentFinish
from langchain_core.exceptions import OutputParserException
from typing import Union, List
from langchain.tools.base import BaseTool
from pydantic import ConfigDict

# 初始化本地Ollama模型
llm = ChatOllama(
    model="deepseek-r1:1.5b",
    base_url="http://localhost:11434",
    temperature=0.0
)

# 创建工具列表
tools = [PythonREPLTool()]

# 自定义输出解析器
class CustomOutputParser(AgentOutputParser):
    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        match = re.search(r'({.*})', text, re.DOTALL)
        if match:
            json_output = match.group(1)
            try:
                import json
                data = json.loads(json_output)
                # 确保返回值中包含 'output' 键
                return AgentFinish(
                    return_values={"output": data},
                    log=text
                )
            except json.JSONDecodeError:
                pass
        raise OutputParserException(f"无法解析输出: {text}")

# 自定义提示模板类，继承自PromptTemplate
class CustomPromptTemplate(PromptTemplate):
    tools: List[BaseTool]
    model_config = ConfigDict(extra='allow')  # 允许额外的参数

    def __init__(self, template, tools, input_variables):
        super().__init__(template=template, input_variables=input_variables, tools=tools)
        self.tools = tools

    def format(self, **kwargs):
        tool_str = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_list"] = tool_str
        return super().format(**kwargs)

# 定义模板
template = """{input}
{intermediate_steps}
{memory_key}"""
memory_key = "chat_history"

# 配置内存
memory = ConversationBufferMemory(memory_key=memory_key, return_messages=True, output_key='output')

# 自定义提示模板
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps", memory_key],
)

# 创建LLM链
from langchain.chains import LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# 初始化代理
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=CustomOutputParser(),
    stop=["\nObservation:"],
    allowed_tools=tool_names,
    memory=memory,
)

# 初始化代理执行器
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
)

# 定义任务，明确输出格式和要求
task = '{"instruction": "计算 2 + 3 的结果", "output_format": "JSON", "requirements": "仅输出包含计算结果的JSON，不包含其他任何内容"}'

# 手动处理输入，确保不重复传递 intermediate_steps
inputs = {"input": task, "memory_key": ""}
inputs = {k: v for k, v in inputs.items() if k != 'intermediate_steps'}

result = agent_executor(inputs, return_only_outputs=True)

# 获取并打印结果
output_key = list(result.keys())[0]
final_result = result[output_key]
print(final_result)

C:\Users\33249\AppData\Local\Temp\ipykernel_31884\1938022612.py:16: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(
C:\Users\33249\AppData\Local\Temp\ipykernel_31884\1938022612.py:64: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key=memory_key, return_messages=True, output_key='output')
C:\Users\33249\AppData\Local\Temp\ipykernel_31884\1938022612.py:75: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=



> Entering new AgentExecutor chain...
<think>
首先，我需要计算2加3的结果。根据基本的数学知识，2加上3等于5。

接下来，我要将这个计算结果以JSON格式返回，并且只包含计算结果，不包含任何额外的内容或信息。

最后，确保输出的JSON结构正确，包括“result”字段，其值为5。
</think>

```json
{
  "result": 5
}
```

> Finished chain.
{'result': 5}


In [2]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_ollama import ChatOllama

In [3]:
llm = ChatOllama(
    model="deepseek-r1:1.5b",base_url="http://localhost:11434",
    temperature=0.0 #  设置温度参数为0.0，通常用于控制生成文本的随机性，温度越低生成的文本越确定性
)

In [4]:
# 需要安装wikipedia库 和 numexpr库
"""llm-math 实际上是一个链，它结合了语言模型和计算器来解决数学问题"""
tools = load_tools(["wikipedia", "llm-math"], llm=llm)


In [5]:
agent = initialize_agent(
    tools,  #  传入工具列表，这些工具将在代理执行任务时使用
    llm, #  传入语言模型（Language Model），代理将使用该模型进行推理和决策
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  #  指定代理类型为“零样本反应描述”（Zero-Shot React Description）
    has_extended_color_support=True, #  启用扩展颜色支持，可能用于界面或日志显示
    handle_parsing_errors=True,  # 开启错误处理
    verbose=True #  设置详细模式为True，以便在执行过程中输出更多的调试信息
)

C:\Users\33249\AppData\Local\Temp\ipykernel_31884\2196149598.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


在 `LangChain` 里，借助 `initialize_agent` 函数构建代理时，`agent` 参数用来明确代理的类型。不同的代理类型有着不同的特性与适用场景，下面为你详细介绍 `AgentType.ZERO_SHOT_REACT_DESCRIPTION` 这种代理类型以及其他常见的代理类型。

`AgentType.ZERO_SHOT_REACT_DESCRIPTION` 代理类型

含义
`AgentType.ZERO_SHOT_REACT_DESCRIPTION` 代理类型运用的是“零样本学习”（Zero-Shot Learning）和“反应式行动”（ReACT）框架。
- **零样本学习**：此代理不需要预先训练特定的任务数据，仅依据工具的描述和输入的问题，就能决定使用何种工具来解决问题。也就是说，在面对全新的任务时，它能够利用已有工具的描述信息，快速做出反应并采取行动。
- **反应式行动（ReACT）**：这是一种让代理在推理和行动之间循环的机制。代理会先思考需要做什么，接着采取相应的行动，然后依据行动的结果更新自己的思考，如此循环，直至问题得到解决。

适用场景
适用于那些需要根据不同问题灵活调用各种工具的场景，而且这些工具的描述信息足以让代理理解如何使用它们。例如，在一个集成了搜索引擎、计算器、文件读取等多种工具的系统中，代理可以根据用户提出的问题，如“查询今天的天气并计算温差”，自动决定先使用搜索引擎获取天气信息，再使用计算器计算温差。

其他常见代理类型

`AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION`
- **含义**：这是专门为聊天场景设计的零样本 ReACT 代理。与 `ZERO_SHOT_REACT_DESCRIPTION` 类似，它也能在零样本的情况下使用工具，但更侧重于处理对话式的输入。
- **适用场景**：适用于聊天机器人等交互场景，能够根据用户的聊天内容调用合适的工具进行回复。

`AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION`
- **含义**：这是一种具有对话记忆功能的聊天代理。它不仅可以在对话中使用工具，还能记住之前的对话内容，从而实现更连贯的对话交互。
- **适用场景**：适合需要长期对话交互的场景，如智能客服、虚拟助手等，能够根据历史对话上下文更好地理解用户需求并做出响应。

`AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION`
- **含义**：该代理类型在零样本 ReACT 机制的基础上，支持结构化的工具调用。它能够处理更复杂的工具输入和输出格式。
- **适用场景**：当工具的输入和输出具有特定的结构和格式要求时，这种代理类型会更加适用，例如调用具有复杂参数的 API 接口。

总结
在 `initialize_agent` 函数中指定代理类型，能够让你根据具体的应用场景和需求，选择最合适的代理行为模式，从而提高代理解决问题的能力和效率。 

########################################################################################################################

实际上，如果上一块代码没有开启错误处理，那么在使用代理时，可能会出现以下错误：
    ---------------------------------------------------------------------------
OutputParserException                     Traceback (most recent call last)
File c:\Users\33249\anaconda3\envs\xhd\Lib\site-packages\langchain\agents\agent.py:1354, in AgentExecutor._iter_next_step(self, name_to_tool_map, color_mapping, inputs, intermediate_steps, run_manager)

这是因为，这个错误信息表明在使用Langchain的代理（Agent）时，输出解析器无法按照预期解析语言模型（LLM）的输出。具体分析如下：
1. **错误类型及原因**：
    - 核心错误是`OutputParserException`，具体错误信息为`Could not parse LLM output`，即无法解析LLM的输出。从给出的LLM输出内容来看，模型输出的是一段思考过程和最终答案，但输出格式不符合当前使用的`MRKLOutputParser`解析器所期望的格式。`MRKLOutputParser`解析器通常期望输出遵循特定的格式，比如包含明确的`Action`和`Action Input`字段 ，但实际输出中并没有这些字段，而是一段自然语言描述的计算过程和答案。
    - 在处理`OutputParserException`异常的过程中，又引发了`ValueError`，提示为了将这个错误传递回代理并让其重试，可以在创建`AgentExecutor`时传入`handle_parsing_errors=True`参数。
2. **相关代码位置及执行逻辑**：
    - 出错的代码位于`AgentExecutor._iter_next_step`方法中，在调用`self._action_agent.plan`时触发了输出解析错误。`Agent.plan`方法中使用`self.output_parser.parse`来解析LLM的输出，而这里使用的`MRKLOutputParser.parse`方法在解析时发现输出不符合预期格式，从而抛出异常。
    - 在`AgentExecutor`执行代理的过程中，它会不断循环调用`_iter_next_step`方法来决定下一步的行动。当输出解析失败时，如果没有设置`handle_parsing_errors=True`，就会抛出`ValueError`异常并终止执行。
3. **解决建议**：
    - **调整解析器或输出格式**：如果可能，修改输出解析器，使其能够处理这种自然语言描述的输出，或者调整LLM的提示（prompt），让模型输出符合现有解析器期望的格式。例如，如果使用的是自定义解析器，可以根据实际输出结构修改解析逻辑；如果是使用预定义的解析器，可以参考文档调整提示，让模型按要求输出。
    - **开启错误处理并调整代理行为**：按照`ValueError`中的提示，在创建`AgentExecutor`时传入`handle_parsing_errors=True`参数。这样当出现解析错误时，代理可以尝试其他方式来处理，而不是直接报错终止。不过开启这个参数后，需要确保代理在遇到解析错误时的重试逻辑是合理有效的。
    - **使用更合适的工具或框架**：考虑使用`LangGraph`等更灵活的框架来构建代理逻辑，这些框架在处理输出和错误时可能更具优势，能够更好地适应复杂的输出格式。此外，也可以参考`Langchain`官方文档中关于结构化输出（structured output）的内容，尝试使用工具调用（tool calling）等技术，让模型输出更易于解析的结构化数据。 

    

In [6]:
# 这里实际上采用的是Chain的_call_方法，而不是AgentExecutor的run方法，所以不需要output_parser
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
<think>
Okay, so I need to figure out what 25% of 300 is. Hmm, percentages can sometimes be a bit tricky, but let me break it down step by step.

First, I know that "percent" means "per hundred." So, when we say 25%, it's the same as saying 25 per 100 or 25/100. That makes sense because percentages are just a way to express parts of a whole, where the whole is 100.

Now, I need to find out what 25% of 300 is. So, essentially, I'm looking for a part of 300 that's equal to 25/100 of it. To do this, I can convert the percentage into a decimal and then multiply it by the number.

Let me write that out: 25% as a decimal is 0.25 because you move the decimal point two places to the left. So, 25 divided by 100 equals 0.25. 

Next, I take this decimal and multiply it by 300. That would be 0.25 multiplied by 300. Let me do that multiplication: 0.25 * 300.

Hmm, multiplying 0.25 by 300... Well, 0.25 is the same as a quarter, so a quarter of 300 should be st

{'input': 'What is the 25% of 300?', 'output': '**  \nThe 25% of 300 is 75.'}

In [7]:
# 使用 invoke 方法替代 __call__ 方法
result = agent.invoke({"input": "What is the 25% of 300?"})
print(result)



> Entering new AgentExecutor chain...
<think>
Okay, so I need to figure out what 25% of 300 is. Hmm, percentages can sometimes be a bit tricky, but let me break it down step by step.

First, I know that "percent" means "per hundred." So, when we say 25%, it's the same as saying 25 per 100 or 25/100. That makes sense because percentages are just a way to express parts of a whole where the whole is 100.

Now, I need to find out what 25% of 300 is. So, essentially, I'm looking for a part of 300 that's equal to 25/100 of it. To do this, I can convert the percentage into a decimal and then multiply it by the number.

Let me write that out: 25% as a decimal is 0.25 because you move the decimal point two places to the left. So, 25 divided by 100 equals 0.25. 

Next, I take this decimal and multiply it by 300. That would be 0.25 multiplied by 300. Let me do that multiplication: 0.25 * 300.

Hmm, multiplying 0.25 by 300... Well, 0.25 is the same as a quarter, so a quarter of 300 should be str

以上的结果，整个过程中，代理执行器运用语言模型的推理能力，通过对百分比概念的理解和数学运算，逐步解决问题，并清晰地呈现出思考过程、计算步骤和最终答案。因为使用的语言模型不一样，所以这个的过程也不一样，可以尝试使用ChatOpenAI模型，看看过程又是怎么样的。

In [8]:
# 因为我们使用的是本地部署的deepseek-r1:1.5b模型，所以这里他并不会联网或者调用搜索引擎的API进行搜索，
# 所以就没有调用API的过程，如果使用的是ChataOpenAI的话，就会调用API进行搜索，
# 这些调用API都是在原本创建AgentExecutor的时候设置的零样本反应中进行的
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
<think>
Okay, so I need to figure out what book Tom M. Mitchell wrote. Let me start by recalling some of his areas of expertise. He's a computer scientist and a professor at CMU, which is a well-known institution for AI research. 

I remember that Mitchell has written several influential books in the field of artificial intelligence. One of them comes to mind: "Machine Learning". That sounds familiar because machine learning is a big area he contributed to. I think he wrote this book as his Ph.D., and it's considered a foundational text in the field.

Wait, let me double-check that. Yes, Mitchell co-wrote "Artificial Intelligence" with David S.AML. But that was published later, so maybe that's another one. However, since the question is about what he wrote specifically, I'm pretty sure "Machine Learning" is his most notable work.

I should also consider if there are other books he might have written. He has a book called "Learning from Data", whi

In [9]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [10]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [11]:
agent.invoke(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...
<think>
Okay, so I need to sort these customers by their last name and then first name. The list given is [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]. 

First, I should understand what sorting by last name and then first name means. It's a stable sort where the primary key is the last name, and the secondary key is the first name. So, if two people have the same last name, they will be sorted by their first names alphabetically.

Looking at the list, each sublist has a first name and a last name. I'll need to extract all the last names to determine the primary sort order. The last names in the list are: Harrison, Lang, Dolly, Elle, Geoff, trance, Jen.

I should arrange these last names in alphabetical order. Let's see:

Dolly comes first because it starts with 'D'. Then Jen, which is next. Then Geoff, followed by Harrison. After that, trance, then Lang,

{'input': "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
 'output': "The sorted list is [['Dolly', 'Too'], ['Jen', 'Ayai'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Trance', 'Former'], ['Lang', 'Chain'], ['Elle', 'Elem']]."}

这里的输入不同，一个是代码，一个是文字，这可能是由以下原因导致：
### 模型特性与默认设置
- **ChatOpenAI**：OpenAI 的模型在设计上常被用于处理代码相关任务，对代码的理解和生成能力经过大量代码数据训练，在处理编程任务时倾向于接收和输出代码形式指令 。其工具使用规范等也鼓励以代码方式实现功能，所以在创建 Python 代理这类编程任务场景，自然采用代码输入方式。 
- **ollama本地模型**：ollama 部署的本地模型可能在训练数据构成、指令遵循模式等方面有差异。如果训练数据中文字描述性的编程任务案例较多，或者其默认的交互引导倾向于自然语言，就会以文字形式表达任务处理逻辑，而非直接代码形式。

### 调用配置与工具适配
- **ChatOpenAI**：在 langchain 框架中与 ChatOpenAI 集成时，相关工具（如 Python REPL 工具）的调用配置可能预设为让模型以代码方式调用工具，以便精准执行代码逻辑。 
- **ollama本地模型**：ollama 本地模型在 langchain 中的集成方式、工具调用配置不同。也许其适配的工具调用流程更强调自然语言描述，先让模型用文字阐述思路，再转化为实际操作（可能通过后续步骤解析文字生成代码执行） 。 

### 模型训练数据差异
- **ChatOpenAI**：训练数据涵盖海量公开代码库、编程论坛问答等代码类数据，使其对代码的处理能力强，能更好理解和生成代码完成任务。 
- **ollama本地模型**：若训练数据中代码类数据占比少，自然语言描述编程任务的数据占比多，就会导致在创建 Python 代理这类任务中，倾向用文字表达而非代码。 

In [12]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an

C:\Users\33249\AppData\Local\Temp\ipykernel_31884\3881924213.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run(f"""Sort these customers by \


[llm/end] [chain:AgentExecutor > chain:LLMChain > llm:ChatOllama] [752.34s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "<think>\nOkay, so I need to sort these customers by their last name and then first name. The list given is [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]. \n\nFirst, I should understand what sorting by last name and then first name means. It's a stable sort where the primary key is the last name, and the secondary key is the first name. So, if two people have the same last name, they will be sorted by their first names alphabetically.\n\nLooking at the list, each sublist has a first name and a last name. I'll need to extract all the last names to determine the primary sort order. The last names in the list are: Harrison, Lang, Dolly, Elle, Geoff, trance, Jen, Ayai.\n\nI should arrange these last names alphabetically. Let's see:\n\nA comes be

In [13]:
from langchain.agents import tool
from datetime import date

@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [14]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [15]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
<think>
Okay, so I need to figure out what today's date is using the tools provided. Let me start by recalling the available tools. There's Wikipedia as a wrapper, a calculator for math problems, and time to get today's date.

First, I'll check if Wikipedia has an entry on today's date. I know that Wikipedia does have a lot of information, but I'm not sure about the current date. Maybe I can search it using "today" or "what is today." If I find something, that would be helpful. But since I don't have access to external sources right now, I'll assume that Wikipedia doesn't have an entry on today's date.

Next, I'll try the time tool. The input should be an empty string because it always returns today's date. So I'll use $TIME and see what it gives me. If the time tool is reliable, it should return today's date accurately.

If both tools don't provide the information I need, maybe there's another way to get the current date. But based on the instru